In [1]:
import cv2 as cv
from utils import *


# Window adjustment==========================
w_width  = 540
w_height = 400

cv.namedWindow("Raw",cv.WINDOW_NORMAL)
cv.namedWindow("Model",cv.WINDOW_NORMAL)
cv.namedWindow("Hough",cv.WINDOW_NORMAL)

cv.resizeWindow("Raw", w_width, w_height)
cv.resizeWindow("Model", w_width, w_height)
cv.resizeWindow("Hough", w_width, w_height)

cv.moveWindow("Hough", w_width, w_height);
cv.moveWindow("Model", 0, w_height);
cv.moveWindow("Raw", w_width, 0);
# Window adjustment==========================

# Parameters=================================
# Hough Lines
h_rho = 0.8
h_theta = np.pi / 180
h_tresh =70 	# Minimum number of pixed in a line
h_lines = None
h_srn = 0 	# divisor for rho	
h_stn = 0  	# divisor for theta

# Canny
c_t1 = 50
c_t2 = c_t1*3
# Parameters=================================

In [2]:
#Process model image=========================
model_img = cv.imread("model.jpg")
model_gray = cv.cvtColor(model_img, cv.COLOR_BGR2GRAY)    
model_canny = cv.Canny(model_gray, c_t1, c_t2)
model_lines = cv.cvtColor(model_canny, cv.COLOR_GRAY2BGR)
source_lines = cv.HoughLines(model_canny, 1.1, h_theta, 247, h_lines, h_srn, h_stn)

# Add the lines that we got from Hough routine to the model_lines frame
# Also convert polar coordinates to euclidean coordinats for k-means clustering
cartesian_lines = []
if source_lines is not None:
	for i in range(0, len(source_lines)):
		pt1, pt2 = add_line(source_lines, i, model_lines)
		cartesian_lines.append([[pt1[0],pt1[1]], [pt2[0],pt2[1]]])

# Find all junctions and perform k-means
# K-means cluster size and hough parameters are decided via
# Trial and error and explained in the project report
all_intersections = junctions(cartesian_lines)
cluster_centers = kmeans_centers(all_intersections , 10)

# Mark cluster centers on the modal lines
mark_all(cluster_centers, model_lines)

# Write the image to disk to show on the report :)
# cv.imwrite("model_lines.png", model_lines)

# Calculate corners
m_top,m_bot = get_model_corners(cluster_centers)
print("Cluster centers:", cluster_centers)
print("Points on top:", m_top)
print("Points on bot:", m_bot)
model_joined_topbot = m_top + m_bot
#Process model image=========================

Cluster centers: [[941.5 527. ]
 [202.   45. ]
 [202.  527. ]
 [941.5  45. ]
 [496.5  45. ]
 [496.5 527. ]
 [791.  527. ]
 [ 49.5 527. ]
 [ 49.5  45. ]
 [791.   45. ]]
Points on top: [[49, 45], [941, 45]]
Points on bot: [[49, 527], [941, 527]]


In [3]:

# Initilize warped image incase the program fails to calculate one
# at the first frame
warped_img = []
cap = cv.VideoCapture(0)

while(True):
	_ , frame_raw = cap.read()

	# Raw frame > gray scaled > Canny > Hough Lines====
	frame_gray = cv.cvtColor(frame_raw, cv.COLOR_BGR2GRAY)    
	frame_canny = cv.Canny(frame_gray, c_t1, c_t2)
	frame_lines = cv.cvtColor(frame_canny, cv.COLOR_GRAY2BGR)
	lines = cv.HoughLines(frame_canny, h_rho, h_theta, h_tresh, h_lines, h_srn, h_stn)
	# =================================================
	
	cartesian_lines = []
	if lines is not None:
		for i in range(0, len(lines)):
			pt1, pt2 = add_line(lines, i, frame_lines)
			cartesian_lines.append([[pt1[0],pt1[1]], [pt2[0],pt2[1]]])
 
	# Find all junctions and perform k-means and mark them
	junction_points = junctions(cartesian_lines)
	cluster_centers = kmeans_centers(junction_points, 37)
	mark_all(junction_points, frame_lines, rbg=(0,0,255))

	# Check if the clusters center calculation is insufficent
	if type(cluster_centers) is np.ndarray:
		n_noninf = get_noninf(cluster_centers)
		
		# 4 is number of corners, in future this should be generalized 
		# to n points homography
		if(len(n_noninf) >= 4):
			top, bot = get_frame_corners(n_noninf)
			frame_joined_topbot = top + bot
			if(len(frame_joined_topbot) == 4):
				# Calculate the mask for the warping
				mask = np.array([])
				mask , status = cv.findHomography(np.array(frame_joined_topbot), np.array(model_joined_topbot)) 
				print("Mask:\n", mask)
				# Get frame resolution for cv.warpPerspective()
				height, width, channels = model_img.shape
				warped_img = cv.warpPerspective(frame_raw, mask, (width, height))
		# Mark lines on the canny+hough frame
		mark_all(n_noninf, frame_lines)
		print("Clusters: ", len(n_noninf)) # Number of cluster centers on screen


	# Display the frames
	cv.imshow("Raw", frame_raw)
	cv.imshow("Hough", frame_lines)
	# If warped image is calculated(Can be an old frame if the currents couldn't be calculated)
	if (len(warped_img) > 0): 
		cv.imshow("Model", warped_img)

	# 1000/50 = 20 FPS, Quit on keypress Q
	if cv.waitKey(50) & 0xFF == ord('q'): 
		break

# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

D/E:  2.5805696057196736
Points on top: [array([187.26086957, 202.36956522]), array([440.92105263, 303.15789474])]
Points on bot: [array([ 74.68518519, 288.96296296]), array([353.25      , 419.38636364])]
Mask:
 [[ 3.68429692e+00  4.90413046e+00 -1.61328560e+03]
 [-2.35711971e+00  6.01353578e+00 -7.12114230e+02]
 [-1.38562902e-04  2.15378984e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.5707143751271153
Points on top: [array([186.65454545, 201.34545455]), array([441.81395349, 304.18604651])]
Points on bot: [array([ 74.22916667, 289.875     ]), array([354.10526316, 419.44736842])]
Mask:
 [[ 3.41959225e+00  4.44261187e+00 -1.46726969e+03]
 [-2.21037463e+00  5.54782212e+00 -6.44287827e+02]
 [-1.67160171e-04  1.82864890e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.5785728030075075
Points on top: [array([187.19565217, 202.43478261]), array([440.89473684, 303.15789474])]
Points on bot: [array([ 79.40425532, 292.17021277]), array([354.10526316, 419.44736842])]
Mask:
 [[ 3.59308162e+00  4.40743912e

D/E:  2.5698626369956026
Points on top: [array([187.18421053, 202.42105263]), array([439.6969697 , 304.42424242])]
Points on bot: [array([ 78.56521739, 291.7173913 ]), array([353.17948718, 418.84615385])]
Mask:
 [[ 3.44999868e+00  4.28674004e+00 -1.44890829e+03]
 [-2.17357474e+00  5.44283068e+00 -6.35555007e+02]
 [-1.25043331e-04  1.68877024e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.6698653443371967
Points on top: [array([169.3 , 196.55]), array([440.48484848, 303.51515152])]
Points on bot: [array([ 56.4 , 285.95]), array([369.75, 423.5 ])]
Mask:
 [[ 3.22048808e+00  4.20111883e+00 -1.30439970e+03]
 [-2.29479341e+00  5.86756746e+00 -7.03636385e+02]
 [-4.29899339e-04  2.19346776e-03  1.00000000e+00]]
Clusters:  11
D/E:  2.5510922242579666
Points on top: [array([191.98181818, 202.92727273]), array([442.28947368, 303.60526316])]
Points on bot: [array([ 79.36507937, 290.01587302]), array([353.18181818, 419.31818182])]
Mask:
 [[ 3.66205263e+00  4.84425432e+00 -1.61791684e+03]
 [-2.32603692e

D/E:  2.590277548783197
Points on top: [array([187.26086957, 202.36956522]), array([440.89473684, 303.15789474])]
Points on bot: [array([ 79.28301887, 290.98113208]), array([353.18181818, 419.31818182])]
Mask:
 [[ 3.66792341e+00  4.56398127e+00 -1.54328753e+03]
 [-2.24735503e+00  5.73485948e+00 -6.78021726e+02]
 [-4.35918910e-05  1.87390936e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.3856727895734045
Points on top: [array([191.37777778, 202.62222222]), array([417.44230769, 294.78846154])]
Points on bot: [array([ 83.37096774, 291.75806452]), array([353.18181818, 419.31818182])]
Mask:
 [[ 4.44545911e+00  5.57761985e+00 -1.90578396e+03]
 [-2.80164969e+00  6.95041567e+00 -8.03140214e+02]
 [-5.87408430e-04  3.18641490e-03  1.00000000e+00]]
Clusters:  7
D/E:  2.5772626527877027
Points on top: [array([187.26315789, 202.34210526]), array([440.48484848, 303.51515152])]
Points on bot: [array([ 74.68518519, 288.96296296]), array([353.18181818, 419.31818182])]
Mask:
 [[ 3.63801290e+00  4.84167517e+

D/E:  2.5656271569009803
Points on top: [array([187.19565217, 202.43478261]), array([440.92105263, 303.15789474])]
Points on bot: [array([ 73.20833333, 289.47916667]), array([354.07894737, 419.60526316])]
Mask:
 [[ 3.61830947e+00  4.85588496e+00 -1.59173901e+03]
 [-2.33535420e+00  5.95809432e+00 -7.05964006e+02]
 [-1.90087596e-04  2.15099694e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.569864723592207
Points on top: [array([188.12727273, 202.        ]), array([441.69767442, 304.34883721])]
Points on bot: [array([ 74.68518519, 288.96296296]), array([353.25      , 419.38636364])]
Mask:
 [[ 3.51495029e+00  4.69334203e+00 -1.54223561e+03]
 [-2.29509473e+00  5.75647061e+00 -6.69435372e+02]
 [-1.70149742e-04  1.98484257e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.526462955622244
Points on top: [array([194.94871795, 204.20512821]), array([440.48484848, 303.51515152])]
Points on bot: [array([ 88.17391304, 293.97826087]), array([354.02631579, 419.52631579])]
Mask:
 [[ 3.70753757e+00  4.50178942e+0

D/E:  2.387253988190456
Points on top: [array([188.01851852, 202.16666667]), array([415.42857143, 294.25714286])]
Points on bot: [array([ 74.68518519, 288.96296296]), array([353.25      , 419.38636364])]
Mask:
 [[ 4.50916513e+00  6.12096700e+00 -2.00651106e+03]
 [-3.00702061e+00  7.50635701e+00 -8.79837254e+02]
 [-7.85113073e-04  3.73345342e-03  1.00000000e+00]]
Clusters:  7
D/E:  2.6851609833293923
Points on top: [array([168.16666667, 195.95833333]), array([440.92105263, 303.15789474])]
Points on bot: [array([ 57.55555556, 286.44444444]), array([356.18518519, 418.44444444])]
Mask:
 [[ 3.08918809e+00  3.86317643e+00 -1.21382605e+03]
 [-2.04388161e+00  5.25186667e+00 -6.27858096e+02]
 [-1.67151515e-04  1.56961566e-03  1.00000000e+00]]
Clusters:  10
D/E:  2.5552712327976903
Points on top: [array([188.15217391, 202.10869565]), array([440.        , 304.36842105])]
Points on bot: [array([ 79.40425532, 292.17021277]), array([354.10526316, 419.44736842])]
Mask:
 [[ 3.42700456e+00  4.22826333e

D/E:  2.578626274581413
Points on top: [array([187.96296296, 202.22222222]), array([440.46511628, 303.95348837])]
Points on bot: [array([ 79.44680851, 290.82978723]), array([353.36842105, 420.23684211])]
Mask:
 [[ 3.63119446e+00  4.54387883e+00 -1.53450219e+03]
 [-2.26411659e+00  5.69545858e+00 -6.64738376e+02]
 [-7.85757472e-05  1.87961540e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.578260510188703
Points on top: [array([187.19565217, 202.43478261]), array([440.92105263, 303.15789474])]
Points on bot: [array([ 78.56521739, 291.7173913 ]), array([353.41025641, 418.94871795])]
Mask:
 [[ 3.57826559e+00  4.44607477e+00 -1.50376345e+03]
 [-2.19643915e+00  5.60517763e+00 -6.62803525e+02]
 [-7.48464840e-05  1.79438192e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.580569605719668
Points on top: [array([187.26086957, 202.36956522]), array([440.92105263, 303.15789474])]
Points on bot: [array([ 74.68518519, 288.96296296]), array([353.25      , 419.38636364])]
Mask:
 [[ 3.68429692e+00  4.90413046e+00

D/E:  2.6096337969467545
Points on top: [array([183.28205128, 201.07692308]), array([440.92105263, 303.15789474])]
Points on bot: [array([ 68.75, 286.3 ]), array([354.07894737, 419.60526316])]
Mask:
 [[ 3.65235090e+00  5.03416834e+00 -1.61150185e+03]
 [-2.42197634e+00  6.19595789e+00 -7.37523353e+02]
 [-1.85398752e-04  2.31697676e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.5921691776025875
Points on top: [array([187.19565217, 202.43478261]), array([440.92105263, 303.15789474])]
Points on bot: [array([ 79.28301887, 290.98113208]), array([353.25      , 419.38636364])]
Mask:
 [[ 3.68006028e+00  4.57979487e+00 -1.54865992e+03]
 [-2.25293308e+00  5.75552616e+00 -6.81535529e+02]
 [-4.04988835e-05  1.88648433e-03  1.00000000e+00]]
Clusters:  8
D/E:  2.6505294581897365
Points on top: [array([184.32075472, 202.1509434 ]), array([440.04651163, 304.46511628])]
Points on bot: [array([ 77.7962963, 288.       ]), array([325.46478873, 407.45070423])]
Mask:
 [[ 3.39574390e+00  4.23491509e+00 -1.4204420